<h2> Import all libraries

In [96]:
import re
import json
import requests
import pandas as pd
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error,make_scorer
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.model_selection import GridSearchCV, KFold
import pickle


<h3> Data collection

In [34]:
warnings.filterwarnings('ignore')

seasons = [2014,2015,2016,2017,2018,2019,2020,2021,2022]
competitions = ['EPL','La_liga','Serie_A','Bundesliga','Ligue_1','RFPL']

all_data = []
for season in seasons:
    for comp in competitions:
        url = f"https://understat.com/league/{comp}/{season}"
        html_doc = requests.get(url).text

        data = re.search(r"datesData\s*=\s*JSON\.parse\('(.*?)'\)", html_doc).group(1)
        data = re.sub(r'\\x([\dA-F]{2})', lambda g: chr(int(g.group(1), 16)), data)
        data = json.loads(data)

        for d in data:
            all_data.append({
                'season': season,
                'competition': comp,
                'date': d['datetime'][:10], # first ten letters
                'home_team': d['h']['title'],
                'away_team': d['a']['title'],
                'home_goals': d["goals"]["h"],
                'away_goals': d["goals"]["a"],
                'home_xG':d['xG']['h'],
                'away_xG': d['xG']['a'],
                'forecast': list(d.get('forecast', {}).values())
            })

df = pd.DataFrame(all_data)
# Split the forecast list into separate columns
df[['home_win_prob', 'draw_prob', 'away_win_prob']] = df['forecast'].apply(lambda x: pd.Series(x))

# Drop the original forecast column
df = df.drop('forecast', axis=1)

# Drop the games that haven't been played
df = df.dropna(how='any', subset=None)

df.to_csv('xg_model.csv',index=False)

In [64]:
df = pd.read_csv('xg_model.csv')

In [65]:
df.shape

(17892, 12)

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17892 entries, 0 to 17891
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   season         17892 non-null  int64  
 1   competition    17892 non-null  object 
 2   date           17892 non-null  object 
 3   home_team      17892 non-null  object 
 4   away_team      17892 non-null  object 
 5   home_goals     17892 non-null  int64  
 6   away_goals     17892 non-null  int64  
 7   home_xG        17892 non-null  float64
 8   away_xG        17892 non-null  float64
 9   home_win_prob  17892 non-null  float64
 10  draw_prob      17892 non-null  float64
 11  away_win_prob  17892 non-null  float64
dtypes: float64(5), int64(3), object(4)
memory usage: 1.6+ MB


In [67]:
df = df[['home_xG', 'away_xG','home_win_prob', 'draw_prob', 'away_win_prob']]
df[['home_win_prob', 'draw_prob', 'away_win_prob']] = df[['home_win_prob', 'draw_prob', 'away_win_prob']].astype(float)

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17892 entries, 0 to 17891
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   home_xG        17892 non-null  float64
 1   away_xG        17892 non-null  float64
 2   home_win_prob  17892 non-null  float64
 3   draw_prob      17892 non-null  float64
 4   away_win_prob  17892 non-null  float64
dtypes: float64(5)
memory usage: 699.0 KB


<h3> Scale the features - I didnt use this approach after examining the predicted match results

In [7]:
# Define the features to scale
features_to_scale = ['home_xG', 'away_xG']

# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler on the features and transform the data
scaled_features = scaler.fit_transform(df[features_to_scale])

# Replace the original features with the scaled features in the DataFrame
df[features_to_scale] = scaled_features

Split the data into 60 percent train,20% test and 20% validation 

In [69]:
# Shuffle the indices of the data
indices = np.random.permutation(df.shape[0])

# Split the data into train, test, and validation sets
train_indices, test_indices, val_indices = np.split(indices, [int(0.6 * len(indices)), int(0.8 * len(indices))])

train_data = df.iloc[train_indices]
test_data = df.iloc[test_indices]
val_data = df.iloc[val_indices]


In [53]:
print(f"The number of rows in the training set is",len(train_data))
print(f"The number of rows in the test set is",len(test_data))
print(f"The number of rows in the validation set is",len(val_data))

The number of rows in the training set is 10735
The number of rows in the test set is 3578
The number of rows in the validation set is 3579


<h2> Models 

In [9]:
# Select the features and target

X_train = train_data[['home_xG', 'away_xG']]
y_train = train_data[['home_win_prob', 'draw_prob', 'away_win_prob']]

X_test = test_data[['home_xG', 'away_xG']]
y_test = test_data[['home_win_prob', 'draw_prob', 'away_win_prob']]


# Create a decision tree regressor
dt_regressor = DecisionTreeRegressor(max_depth = 3,min_samples_split=5, min_samples_leaf=3, max_features='sqrt',
                                     random_state=42)

# Fit the regressor to the training data
dt_regressor.fit(X_train, y_train)

# Evaluate the model on the test data
y_pred = dt_regressor.predict(X_test)
dt_r2 = r2_score(y_test, y_pred)


# Evaluate the model on the training data
y_pred_train = dt_regressor.predict(X_train)
dt_r2_train = r2_score(y_train, y_pred_train)
mse_train = mean_squared_error(y_train, y_pred_train)
rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)

# Calculate MSE and RMSE on test data
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)


# Print the results
print(f"Training Set R^2 score: {dt_r2_train:.6f}")
print(f"Training Set Mean Squared Error: {mse_train:.6f}")
print(f"Training Set Root Mean Squared Error: {rmse_train:.6f}")

# Print the results
print(f"Test Set R^2 score: {dt_r2:.6f}")
print(f"Test Set Mean Squared Error: {mse:.6f}")
print(f"Test Set Root Mean Squared Error: {rmse:.6f}")

Training Set R^2 score: 0.751901
Training Set Mean Squared Error: 0.010352
Training Set Root Mean Squared Error: 0.099365
Test Set R^2 score: 0.745097
Test Set Mean Squared Error: 0.010852
Test Set Root Mean Squared Error: 0.101762


In [10]:
# Select the features and target
X_train = train_data[['home_xG', 'away_xG']]
y_train = train_data[['home_win_prob', 'draw_prob', 'away_win_prob']]

X_test = test_data[['home_xG', 'away_xG']]
y_test = test_data[['home_win_prob', 'draw_prob', 'away_win_prob']]


# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model on the training data
y_pred_train = model.predict(X_train)
lin_r2_train = r2_score(y_train, y_pred_train)


mse_train = mean_squared_error(y_train, y_pred_train)
rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)

# Calculate MSE and RMSE on test data
y_pred = model.predict(X_test)
lin_r2_test = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)


# Print the results
print(f"Training Set R^2 score: {lin_r2_train:.6f}")
print(f"Training Set Mean Squared Error: {mse_train:.6f}")
print(f"Training Set Root Mean Squared Error: {rmse_train:.6f}")

# Print the results
print(f"Test Set R^2 score: {lin_r2_test:.6f}")
print(f"Test Set Mean Squared Error: {mse:.6f}")
print(f"Test Set Root Mean Squared Error: {rmse:.6f}")

Training Set R^2 score: 0.761854
Training Set Mean Squared Error: 0.006738
Training Set Root Mean Squared Error: 0.082071
Test Set R^2 score: 0.762517
Test Set Mean Squared Error: 0.007080
Test Set Root Mean Squared Error: 0.084090


In [11]:
# Select the features and target
X_train = train_data[['home_xG', 'away_xG']]
y_train = train_data[['home_win_prob', 'draw_prob', 'away_win_prob']]

X_test = test_data[['home_xG', 'away_xG']]
y_test = test_data[['home_win_prob', 'draw_prob', 'away_win_prob']]

# Train an XGBoost Regressor model
xgb_model = XGBRegressor(n_estimators=20, learning_rate=0.1, max_depth=3, random_state=42)
xgb_model.fit(X_train, y_train)

# Make predictions using the XGBoost Regressor model
xgb_preds = xgb_model.predict(X_test)

# Evaluate the model on the training data
y_pred_train = xgb_model.predict(X_train)
xgb_r2_train = r2_score(y_train, y_pred_train)
mse_train = mean_squared_error(y_train, y_pred_train)
rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)

# Calculate MSE and RMSE on test data
xgb_r2_test = r2_score(y_test, xgb_preds)
mse = mean_squared_error(y_test, xgb_preds)
rmse = mean_squared_error(y_test, xgb_preds, squared=False)


# Print the results
print(f"Training Set R^2 score: {xgb_r2_train:.6f}")
print(f"Training Set Mean Squared Error: {mse_train:.6f}")
print(f"Training Set Root Mean Squared Error: {rmse_train:.6f}")

# Print the results
print(f"Test Set R^2 score: {xgb_r2_test:.6f}")
print(f"Test Set Mean Squared Error: {mse:.6f}")
print(f"Test Set Root Mean Squared Error: {rmse:.6f}")

Training Set R^2 score: 0.883586
Training Set Mean Squared Error: 0.003461
Training Set Root Mean Squared Error: 0.058769
Test Set R^2 score: 0.880350
Test Set Mean Squared Error: 0.003688
Test Set Root Mean Squared Error: 0.060645


In [12]:
# Select the features and target
X_train = train_data[['home_xG', 'away_xG']]
y_train = train_data[['home_win_prob', 'draw_prob', 'away_win_prob']]

X_test = test_data[['home_xG', 'away_xG']]
y_test = test_data[['home_win_prob', 'draw_prob', 'away_win_prob']]

# Train the model
model = RandomForestRegressor(max_depth = 3, min_samples_split=5, min_samples_leaf=3,n_estimators=100, random_state=42)
model.fit(X_train, y_train)


# Evaluate the model on the training data
y_pred_train = model.predict(X_train)
rfr_r2_train = r2_score(y_train, y_pred_train)
mse_train = mean_squared_error(y_train, y_pred_train)
rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)

# Calculate MSE and RMSE on test data
y_pred = model.predict(X_test)
rfr_r2_test = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)


# Print the results
print(f"Training Set R^2 score: {rfr_r2_train:.6f}")
print(f"Training Set Mean Squared Error: {mse_train:.6f}")
print(f"Training Set Root Mean Squared Error: {rmse_train:.6f}")

# Print the results
print(f"Test Set R^2 score: {rfr_r2_test:.6f}")
print(f"Test Set Mean Squared Error: {mse:.6f}")
print(f"Test Set Root Mean Squared Error: {rmse:.6f}")

Training Set R^2 score: 0.845388
Training Set Mean Squared Error: 0.006748
Training Set Root Mean Squared Error: 0.079826
Test Set R^2 score: 0.839633
Test Set Mean Squared Error: 0.007155
Test Set Root Mean Squared Error: 0.082205


<h3> Hyperparameter tune and cross validate the best model XGBOOST REGRESSOR
    

In [14]:
%%time

# Define the XGBoost regressor
xgb_reg = XGBRegressor(random_state=42)

# Define the hyperparameters to search over
param_grid = {
    'learning_rate': [0.01, 0.1, 0.5],
    'n_estimators': [10, 50, 100],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'gamma': [0.0, 0.1, 0.5],
    'subsample': [0.5, 0.75, 1.0],
    'colsample_bytree': [0.5, 0.75, 1.0],
}

# Define the scoring metric
scoring = make_scorer(mean_squared_error)

# Define the cross-validation method
cv = KFold(n_splits=10, shuffle=True, random_state=42)

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(xgb_reg, param_grid=param_grid, scoring=scoring, cv=cv)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and corresponding score
print("Best Hyperparameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"Mean Squared Error: {mse:.6f}")
print(f"Root Mean Squared Error: {rmse:.6f}")

Best Hyperparameters:  {'colsample_bytree': 0.5, 'gamma': 0.5, 'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 10, 'subsample': 1.0}
Best Score:  0.07739373519798927
Mean Squared Error: 0.077410
Root Mean Squared Error: 0.277749
CPU times: total: 1d 13h 56min 11s
Wall time: 6h 32min 7s


<h3> Run the best model

In [92]:
# Select the features and target
X_train = train_data[['home_xG', 'away_xG']]
y_train = train_data[['home_win_prob', 'draw_prob', 'away_win_prob']]

X_test = test_data[['home_xG', 'away_xG']]
y_test = test_data[['home_win_prob', 'draw_prob', 'away_win_prob']]

# Train an XGBoost Regressor model
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=10,colsample_bytree=1,
                         min_child_weight=1,subsample=1,gamma=0.5, random_state=42)
xgb_model.fit(X_train, y_train)

# Make predictions using the XGBoost Regressor model
xgb_preds = xgb_model.predict(X_test)

# Evaluate the model on the training data
y_pred_train = xgb_model.predict(X_train)
xgb_r2_train = r2_score(y_train, y_pred_train)
mse_train = mean_squared_error(y_train, y_pred_train)
rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)

# Calculate MSE and RMSE on test data
xgb_r2_test = r2_score(y_test, xgb_preds)
mse = mean_squared_error(y_test, xgb_preds)
rmse = mean_squared_error(y_test, xgb_preds, squared=False)


# Print the results
print(f"Training Set R^2 score: {xgb_r2_train:.6f}")
print(f"Training Set Mean Squared Error: {mse_train:.6f}")
print(f"Training Set Root Mean Squared Error: {rmse_train:.6f}")

# Print the results
print(f"Test Set R^2 score: {xgb_r2_test:.6f}")
print(f"Test Set Mean Squared Error: {mse:.6f}")
print(f"Test Set Root Mean Squared Error: {rmse:.6f}")

Training Set R^2 score: 0.975290
Training Set Mean Squared Error: 0.000592
Training Set Root Mean Squared Error: 0.024251
Test Set R^2 score: 0.972365
Test Set Mean Squared Error: 0.000671
Test Set Root Mean Squared Error: 0.025811


<h2> Predict on the validation set

In [93]:
# Select the features and target for the validation set
X_val = val_data[['home_xG', 'away_xG']]
y_val = val_data[['home_win_prob', 'draw_prob', 'away_win_prob']]

# Make predictions on the validation set using the trained XGBoost Regressor model
xgb_preds_val = xgb_model.predict(X_val)

# Evaluate the model on the validation data
xgb_r2_val = r2_score(y_val, xgb_preds_val)
mse_val = mean_squared_error(y_val, xgb_preds_val)
rmse_val = mean_squared_error(y_val, xgb_preds_val, squared=False)

# Print the results
print(f"Validation Set R^2 score: {xgb_r2_val:.6f}")
print(f"Validation Set Mean Squared Error: {mse_val:.6f}")
print(f"Validation Set Root Mean Squared Error: {rmse_val:.6f}")

Validation Set R^2 score: 0.973300
Validation Set Mean Squared Error: 0.000670
Validation Set Root Mean Squared Error: 0.025797


In [94]:
# correct probabilities as per understats model for below Arsenal v Palace is 0.438,0.2882,0.2738


0.438+0.2882+0.2738


1.0

In [95]:
# Create a new DataFrame with the manual inputs
new_data = pd.DataFrame({
    'home_xG': [1.53568], # Arsenal
    'away_xG': [1.19194] # Palace
})

# Use the model to predict the probabilities
probs = xgb_model.predict(new_data)

# Normalize the predicted probabilities
probs_norm = probs / probs.sum()

# Print the normalized probabilities
print(f"Home win probability: {probs_norm[0][0]:.4f}")
print(f"Draw probability: {probs_norm[0][1]:.4f}")
print(f"Away win probability: {probs_norm[0][2]:.4f}")
print(f"Total probability: {probs_norm.sum():.4f}")

Home win probability: 0.4620
Draw probability: 0.2780
Away win probability: 0.2600
Total probability: 1.0000


Save the trained model 

In [97]:
# Save the trained model to a file
with open('xgb_model.pkl', 'wb') as f:
    pickle.dump(xgb_model, f)

In [ ]:
# Load the saved model from a file
loaded_model = joblib.load('xgb_model.joblib')
# or
# with open('xgb_model.pkl', 'rb') as f:
#     loaded_model = pickle.load(f)

# Use the loaded model for prediction
y_pred = loaded_model.predict(X_test)